In [1]:
import matplotlib.pyplot as plt
import json
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output
from collections import defaultdict

# JSON-Datei initialisieren
JSON_FILE = "tasks.json"

def initialize_json():
    try:
        with open(JSON_FILE, "r") as file:
            data = json.load(file)
    except FileNotFoundError:
        data = {"tasks": [], "completed": [], "revisit": []}
        with open(JSON_FILE, "w") as file:
            json.dump(data, file)
    return data

def load_data():
    with open(JSON_FILE, "r") as file:
        return json.load(file)

def save_data(data):
    with open(JSON_FILE, "w") as file:
        json.dump(data, file)

def add_task(subject, description):
    data = load_data()
    task = {
        "subject": subject,
        "description": description,
        "created_at": datetime.now().isoformat(),
        "completed_at": None
    }
    data["tasks"].append(task)
    save_data(data)

def complete_task(index, revisit=False):
    data = load_data()
    task = data["tasks"].pop(index)
    if revisit:
        task_copy = task.copy()
        task["completed_at"] = None
        task_copy["completed_at"] = datetime.now().isoformat()
        data["completed"].append(task_copy)
        data["revisit"].append(task)
    else:
        task["completed_at"] = datetime.now().isoformat()
        data["completed"].append(task)
    save_data(data)

def remove_revisit_task(index):
    data = load_data()
    data["revisit"].pop(index)
    save_data(data)

def create_ui():
    subject_input = widgets.Text(description="Fach:")
    description_input = widgets.Text(description="Aufgabe:")
    add_button = widgets.Button(description="Aufgabe hinzufügen")
    tasks_output = widgets.Output()
    revisit_output = widgets.Output()
    stats_output = widgets.Output()
    plot_output = widgets.Output()

    def add_task_action(_):
        add_task(subject_input.value, description_input.value)
        subject_input.value = ""
        description_input.value = ""
        update_tasks()
        update_revisit()
        update_statistics()

    add_button.on_click(add_task_action)

    def update_tasks():
        with tasks_output:
            clear_output()
            data = load_data()

            tasks_by_subject = defaultdict(list)
            for idx, task in enumerate(data["tasks"]):
                tasks_by_subject[task["subject"].strip()].append((idx, task["description"]))

            container = []
            for subject, tasks in tasks_by_subject.items():
                rows = [widgets.HTML(f"<h3>{subject}</h3>")]
                row = []
                for idx, description in tasks:
                    task_button = widgets.Button(description=description, layout=widgets.Layout(width="100px", height="40px", padding="5px"))

                    def task_action(b, idx=idx):
                        show_task_options(idx)

                    task_button.on_click(task_action)
                    row.append(task_button)
                    if len(row) >= 10:
                        rows.append(widgets.HBox(row))
                        row = []

                if row:
                    rows.append(widgets.HBox(row))
                container.append(widgets.VBox(rows))

            display(widgets.VBox(container))

    def show_task_options(task_index):
        with tasks_output:
            clear_output()
            task_options = widgets.HBox()

            done_button = widgets.Button(description="Erledigt", button_style="success")
            revisit_button = widgets.Button(description="Nochmal", button_style="warning")
            cancel_button = widgets.Button(description="Abbrechen")

            def mark_done(_):
                complete_task(task_index)
                update_tasks()
                update_revisit()
                update_statistics()

            def mark_revisit(_):
                complete_task(task_index, revisit=True)
                update_tasks()
                update_revisit()
                update_statistics()

            def cancel_action(_):
                update_tasks()

            done_button.on_click(mark_done)
            revisit_button.on_click(mark_revisit)
            cancel_button.on_click(cancel_action)

            task_options.children = [done_button, revisit_button, cancel_button]
            display(task_options)

    def update_revisit():
        with revisit_output:
            clear_output()
            data = load_data()
            revisit_by_subject = defaultdict(list)

            for idx, task in enumerate(data["revisit"]):
                revisit_by_subject[task["subject"].strip()].append((idx, task["description"]))

            container = []
            for subject, tasks in revisit_by_subject.items():
                rows = [widgets.HTML(f"<h3>{subject}</h3>")]
                row = []
                for idx, description in tasks:
                    revisit_button = widgets.Button(description=description, layout=widgets.Layout(width="100px", height="40px", padding="5px"))

                    def remove_action(b, idx=idx):
                        remove_revisit_task(idx)
                        update_revisit()
                        update_statistics()

                    revisit_button.on_click(remove_action)
                    row.append(revisit_button)
                    if len(row) >= 10:
                        rows.append(widgets.HBox(row))
                        row = []

                if row:
                    rows.append(widgets.HBox(row))
                container.append(widgets.VBox(rows))

            if container:
                display(widgets.VBox(container))
            else:
                display(widgets.HTML("<h4>Keine Aufgaben zum Wiederholen.</h4>"))

    def update_statistics():
        with stats_output:
            clear_output()
            stats, percentage_stats, total = get_statistics()

        with plot_output:
            clear_output()
            plot_statistics_combined()

    update_tasks()
    update_revisit()  # Sicherstellen, dass die zu wiederholenden Aufgaben beim Laden angezeigt werden
    update_statistics()

    display(widgets.VBox([
        subject_input, description_input, add_button, tasks_output, stats_output, plot_output, revisit_output
    ]))

def get_statistics():
    data = load_data()
    total_tasks = len(data["tasks"]) + len(data["completed"]) + len(data["revisit"])
    if total_tasks == 0:
        return {}, {}, 0

    stats = {}
    total_subject_tasks = {}

    for task in data["tasks"] + data["completed"] + data["revisit"]:
        subject = task["subject"]
        total_subject_tasks[subject] = total_subject_tasks.get(subject, 0) + 1

    for task in data["completed"]:
        subject = task["subject"]
        stats[subject] = stats.get(subject, 0) + 1

    percentage_stats = {
        subject: (stats.get(subject, 0) / total_subject_tasks[subject]) * 100
        for subject in total_subject_tasks
    }

    return stats, percentage_stats, total_tasks

def plot_statistics_combined():
    stats, percentage_stats, total_tasks = get_statistics()
    daily_stats = get_tasks_per_day_by_subject()

    if not stats and not daily_stats:
        print("Keine Daten für die Diagramme verfügbar.")
        return

    subjects = list(stats.keys())
    values = list(stats.values())
    percentages = [percentage_stats[subj] for subj in subjects]

    color_mapping = get_color_mapping(subjects)
    subject_colors = [color_mapping[subj] for subj in subjects]

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # Abgeschlossene Aufgaben pro Fach
    axes[0].bar(subjects, values, color=subject_colors)
    axes[0].set_title("Abgeschlossene Aufgaben pro Fach")
    axes[0].set_xlabel("Fach")
    axes[0].set_ylabel("Anzahl Aufgaben")
    axes[0].tick_params(axis='x', rotation=0)

    # Prozentualer Fortschritt
    axes[1].bar(subjects, percentages, color=subject_colors, alpha=0.8)
    axes[1].set_title("Prozentualer Fortschritt pro Fach")
    axes[1].set_xlabel("Fach")
    axes[1].set_ylabel("Fortschritt (%)")
    axes[1].set_ylim(0, 100)
    axes[1].tick_params(axis='x', rotation=0)

    # Tagesplot: Aufgaben pro Tag
    dates = sorted(daily_stats.keys())
    subjects_per_day = {subj: [] for subj in subjects}

    for date in dates:
        for subj in subjects:
            subjects_per_day[subj].append(daily_stats.get(date, {}).get(subj, 0))

    bottom = [0] * len(dates)
    for subj in subjects:
        axes[2].bar(dates, subjects_per_day[subj], bottom=bottom,
                    color=color_mapping[subj], label=subj)
        bottom = [sum(x) for x in zip(bottom, subjects_per_day[subj])]

    axes[2].set_title("Aufgaben pro Tag und Fach")
    axes[2].set_xlabel("Datum")
    axes[2].set_ylabel("Anzahl Aufgaben")
    axes[2].tick_params(axis='x', rotation=0)
    axes[2].legend(title="Fächer", bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.tight_layout()
    plt.show()

def get_tasks_per_day_by_subject():
    data = load_data()
    daily_stats = defaultdict(lambda: defaultdict(int))

    for task in data["completed"]:
        date = datetime.fromisoformat(task["completed_at"]).strftime("%Y-%m-%d")
        subject = task["subject"]
        daily_stats[date][subject] += 1

    return daily_stats

def get_color_mapping(subjects):
    color_palette = plt.cm.tab10.colors
    return {subject: color_palette[i % len(color_palette)] for i, subject in enumerate(subjects)}

initialize_json()
create_ui()